In [12]:
import os
from pathlib import Path

import puppets_data
import ConnectionGraphX
import GraphX

import numpy as np
import pandas as pd
import scipy
import networkx as nx

from proxi.algorithms.knng import get_knn_graph
from proxi.utils.misc import save_graph, save_weighted_graph
from proxi.utils.process import *
from proxi.utils.distance import abs_correlation

In [13]:
p = "C:/Users/Sawyer/Documents/python_projects/connection_graph_wasserstein/PuppetsData/"

## Just for time and sanity, using n = 100 while I test things.

X = puppets_data.puppets_data(p, n = 1000)


X.shape =  (1000, 100)


In [14]:
N_IMAGES = X[0].shape[0]
N_NEAREST_NEIGHBORS = 28   # Again, just for time.
DIST = abs_correlation    # Used for kNN calculations

df = pd.DataFrame(X[0])
nodes, a = get_knn_graph(df, k=N_NEAREST_NEIGHBORS,  metric=DIST)


G:\My Drive\python_projects\PyGraphX\venv\lib\site-packages\scipy\sparse\_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [ ]:
INTRINSIC_DIMENSION = 4
print(a.toarray().shape)
g = ConnectionGraphX.ConnectionGraphX(a.toarray(), INTRINSIC_DIMENSION)

neighborhoodPCA = []

for i in range(N_IMAGES):
    features = []
    for node in sorted(a[i].indices):
        features.append(X[0][node])
    neighborhoodPCA.append(puppets_data.do_pca(np.array(features), INTRINSIC_DIMENSION))

for node in range(N_IMAGES):
    iterNodes = [neighbor for neighbor in a[node].indices if neighbor > node]
    for neighbor in iterNodes:
        pcaDataNode = neighborhoodPCA[node]
        pcaDataNeighbor = neighborhoodPCA[neighbor]
        if len(pcaDataNode) < len(pcaDataNeighbor):
            for i in range(len(pcaDataNeighbor) - len(pcaDataNode)):
                pcaDataNode = np.vstack([pcaDataNode, [0] * INTRINSIC_DIMENSION])
        if len(pcaDataNode) > len(pcaDataNeighbor):
            for i in range(len(pcaDataNode) - len(pcaDataNeighbor)):
                pcaDataNeighbor = np.vstack([pcaDataNeighbor, [0] * INTRINSIC_DIMENSION ])

        # Scipy has its own procrustes implementation

        O, _ = scipy.linalg.orthogonal_procrustes(pcaDataNode, pcaDataNeighbor)
        g.setEdgeConnection((node, neighbor), O)



(1000, 1000)


In [ ]:
h = nx.Graph(a)
nx.draw_spectral(h)